TEST BOUNDING BOX

In [10]:
import numpy as np
import cv2  
from mss import mss

# Create bounding box
bounding_box = {'top': 450, 'left': 900, 'width': 120, 'height': 200}
# bounding_box = {'top': 285, 'left': 450, 'width': 896, 'height': 287}

# Instantiate screen capture
sct = mss()

# While true, show screen capture, wait until q is pressed to stop
while True:
    sct_img = sct.grab(bounding_box)
    cv2.imshow('screen', np.array(sct_img))

    key = cv2.waitKey(1)
    if key==ord('q'):
        break
        
cv2.destroyAllWindows()

AUTO FISHING

lower_red - [185, 10, 10]
upper_red - [225, 50, 50]

underwater_lower_red - [70, 50, 100]
underwater_upper_red - [100, 75, 125]

In [7]:
import numpy as np
import cv2  
from mss import mss
import time
import mouse

# Instantiate
sct = mss()

# Bounds for color of lure above water
lower_red = np.array([185, 10, 10])
upper_red = np.array([225, 50, 50])
# Bounds for color of lure below water
underwater_lower = np.array([70, 50, 100])
underwater_upper = np.array([100, 75, 125])

# Line cast timer variables | Not currently used
linecast_start = 0
linecast_end = 0
linecast_time = 10
elapsed_linecast_time = 0
line_cast = False
# Bite timer variables
bite_start = 0
bite_end = 0
bite_min_time = .35
elapsed_bite_time = 0
bite = False
# Recast timer variables
recast_start = 0
recast_end = 0
recast_time = .85
elapsed_recast_time = 0
recast = False

while True:
    # Screen Capture
    sct_img = sct.grab(bounding_box)
    
    # Convert to RGB color because everything else is scary
    img_rgb = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2RGB)
    
    # Check to see if color in range is found | Lure is on screen
    above_mask = cv2.inRange(img_rgb, lower_red, upper_red)
    under_mask = cv2.inRange(img_rgb, underwater_lower, underwater_upper)
    
    # Detect if color is in capture
    redCount = cv2.findContours(above_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    underwaterRedCount = cv2.findContours(under_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    
    # Lure spotted above water
    if len(redCount) > 0:
        if line_cast == False:
            line_cast = True
            print("Line Cast")
    else:
        line_cast = False
#         if linecast_start == 0:
#             linecast_start = time.time()
#         else:
#             linecast_end = time.time()
#             elapsed_linecast_time = linecast_end - linecast_start
            
#             if elapsed_linecast_time > linecast_time:
#                 mouse.click('right')
#                 linecast_start = 0
        
        
    # Hook above water underwater     
    if len(underwaterRedCount) > 0:
        if bite == False:
            bite = True
            bite_start = time.time()
        else:
            bite_end = time.time()
            elapsed_bite_time = bite_end - bite_start
            
            if elapsed_bite_time > bite_min_time:
                print("BITE")
                mouse.click('right')
                bite = False
                recast = True                
    else:
        bite = False
    
    # Lure has been reeled in, time to recast
    if recast:
        if recast_start == 0:
            recast_start = time.time()
        else:
            recast_end = time.time()
            elapsed_recast_time = recast_end - recast_start
            
            if elapsed_recast_time > recast_time:
                mouse.click('right')
                recast = False
                recast_start = 0
    
    # Show all 3 screen captures
    cv2.imshow('screen', np.array(sct_img))
    cv2.imshow('Line Cast', above_mask)
    cv2.imshow('Bite', under_mask)

    # Wait for q to be pressed
    key = cv2.waitKey(1)
    if key==ord('q'):
        break
        
# Reset all screen captures
cv2.destroyAllWindows()

Line Cast
BITE
Line Cast
BITE
Line Cast
Line Cast
BITE
Line Cast
BITE
Line Cast
